# Reproduce NMR HDX Figure

Inputs:

+ `peak-files/` contains sparky-style peak lists for hA9 and hA9/M63F
+ `peak-file-index.xlsx` maps those peak files to their time points
+ `structure/` contains a pdb file (`1irj.pdb`) for mapping to structure
   and a pymol script (`generate-structure-plots.pml`) for generating the
   structure figure. 
  
To generate the figure, run this notebook, and then use pymol to run 
`generate-structure-plots.pml`

## Configure environment

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.optimize import curve_fit

## Define functions

In [ ]:
def get_file_time_index(excel_file,dir_base):
    """
    Take an excel file with the columns:
    
    File #: number of file holding peaks at this time point
    Protein: protien name for this file
    Timepoint (minutes): time point in minutes
    
    dir_base is directory holding the peaks files. 
    
    Returns dataframe with protein, filename, and time columns
    """

    protein = []
    filename = []
    time = []

    df = pd.read_excel(excel_file)
    for i in range(len(df)):
        row = df.iloc[i,:]
        if np.isnan(row["File #"]):
            continue

        prot = row["Protein"]

        protein.append(prot)
        filename.append(os.path.join(dir_base,
                                     prot.lower(),
                                     "{:d}.list".format(int(row['File #']))))
        time.append(row["Timepoint (minutes)"])

    return pd.DataFrame({"protein":protein,"time":time,"filename":filename})


def load_peak_files(df):
    """
    Load peak files corresponding to HSQC vs. time into a single data frame. 
    df is a dataframe generated by get_file_time_index. 
    """
    
    final_data = pd.DataFrame()
    
    n = 0
    for i in range(len(df)):
        row = df.iloc[i]
        
        # read in data, get rid of duplicates/empties
        data = pd.read_fwf(row.filename)
        data = data.drop_duplicates()
        data = data.dropna()
        data = data.drop(data[data["Assignment"]== "?-?"].index)
        data = data.drop(data[data["Assignment"]== "W88NE1-HE1"].index)        

        data = data.reset_index(drop=True)
        
        # add residue number and timepoint, drop all columns except res # and intensity

        x = [i.split("N-H")[0] for i in data.Assignment.values]
        x = [int(i[1:]) for i in x]
        data["residue"] = x
        data["time"] = row.time
        data["protein"] = row.protein
        data = data.drop(["Assignment", "w1", "w2"], axis=1)
        n+=1

        final_data = pd.concat([final_data, data])

    final_data = final_data.rename(columns={"Data Height":"height"})
        
    return final_data

def hdx_fitter_aic(input_data,
                   try_to_fit_cutoff=1.0,
                   slow_cutoff=1.0,
                   num_across=9,
                   ylim=None,xlim=None,
                   colors=["black","red"],
                   secondary_colors=["gray","pink"],
                   manual_mapping=None,
                   plot_zero_points=False):
    """
    Take a data frame holding proteins, residues, times, and heights and then
    fit the decay constant for each protein/residue pair.  Generates an array
    of subplots, one for each residue.  The behavior of the residue in a each 
    protein is plotted as a series on the subplot.  The program fits a linear,
    exponential, and stretched exponential model to each series, and then 
    decides which fit is justified using an AIC test.  The zero time point is
    not used in the fit. 
    
    input_data: output of load_peak_file.  expected to have colummns with
                protein, residue, time, and height
    try_to_fit_cutoff: only try to fit a curve where residue has at least one
                       peak for t > 0 with height above try_to_fit_cutoff. 
                       height is given in millionths of intensity, meaning 
                       try_to_fit_cutoff=1.0 means peaks must be above 1e6.
    slow_cutoff: 
    num_across: how many columns in subplot.  Number of rows will be determined
                automatically to accomodate all residues.
    xlim, ylim: xlim and ylim (each a 2-length tuple) indicating the limits on
                on each subplot
    colors: colors for proteins.  proteins will be plotted in alphabetical
            order, so the first color in the list will be for the first
            alphabetical protein, etc. 
    secondary_colors: colors for proteins that are assigned as "fast" or "slow"
                      and thus not fit explicitly.
    manual_mapping: dictionary with form {protein_name:{residue_number:assign}}
                    where "assign" is either fast or slow.  This lets the user
                    manually specify that a given protein/residue should be 
                    forced to be fast or slow.  This assumes the fitter found
                    the best model for that protein/residue pair was linear, so
                    not cheating and calling an exponential fit fast or slow.
    plot_zero_points: plot lines indicating position of zero points
                    
    Returns fig, ax, and dataframe holding fit results. 
    """
    
    # -------------------------------------------------------------------------
    # Local functions for performing fits
    
    def line(x, b):
        return b
    
    def decay(x, a, k):
        return a * np.exp(-k * x)

    def decay_stretch(x, a, k, beta):
        return a * np.exp(-(k * x)**beta)
        
    def get_aic(model_vals,Y,k):
        """
        model_vals: model values at points correspodning to Y
        Y: Y points
        k: number of fit parameters
        
        AICc due to small sample size:
        https://en.wikipedia.org/wiki/Akaike_information_criterion
        """
        
        sigma2 = np.std(Y)**2        
        lnL = -0.5*(np.sum((Y - model_vals)**2/sigma2 + np.log(sigma2)))
        
        n = len(Y)

        aic = 2*k - 2*lnL
        
        corr = (2*k**2 + 2*k)/(n - k - 1)

        return aic + corr
        
    
    # -------------------------------------------------------------------------
    # Preprocess data frame
    
    all_data = input_data.copy()
    
    # Rescale data so time is in seconds and height is in millions of units
    all_data["time"] = all_data["time"]*60
    all_data["height"] = all_data["height"]/1e6
    
    # This will hold fractional height relative to the zero time point
    all_data["fx"] = all_data["height"]
    
    # Sorted lists of proteins and residues
    proteins = list(all_data.protein.unique())
    proteins.sort()
    
    resid_list = list(all_data.residue.unique())
    resid_list.sort()
    
    # For each protein...
    for p in proteins:        
        
        # Data for that protein
        x = all_data[all_data.protein == p]
        
        # Get min and average height for this protein 
        min_value = np.nanmin(x["fx"])
        avg_zero = np.nanmean(x[x.time == 0].height)
        
        # For each residue
        for r in resid_list:            
            
            # Data for residue in that protein
            y = x[x.residue == r]
            
            # If such data exists
            if len(y) > 0:
                
                # Get zero point
                z = y[y.time == 0]
                
                # If no zero point for this residue, assign start height
                # to the average height at zero
                if len(z) > 0:
                    start_height = z.height.iloc[0]
                else:
                    start_height = avg_zero
                
                # If start height is nan or negative, assign to average
                if np.isnan(start_height) or start_height <= 0:
                    start_height = avg_zero
                
                # Mask on all data that matches the protein and residue
                mask = np.logical_and(all_data.protein == p,
                                      all_data.residue == r)
                
                # Normalize data to be between zero and one
                v = (all_data.loc[mask,"fx"] - min_value)/(start_height - min_value)
                
                # Update main data frame
                all_data.loc[mask,"fx"] = v
    
    # Separate zero points from the main data frame
    zero_points = all_data[all_data.time == 0]
    all_data = all_data[all_data.time != 0]
                
    # -------------------------------------------------------------------------
    # Construct plot object

    # Figure out how big to make the plot array
    num_resid = len(resid_list)
    num_down = num_resid // num_across + int(num_resid % num_across != 0)
    
    # Create fig and ax array objects
    fig, ax = plt.subplots(num_down,num_across,figsize=(8,10.5))
    
    # Make 1000 point array for drawing smooth lines
    x_span = np.linspace(0,np.max(all_data["time"]),1000)

    # If xlim is not specified, figure out useful xlim
    if xlim is None:
    
        x_min = np.nanmin(all_data["time"])
        x_max = np.nanmax(all_data["time"])
        x_offset = (x_max - x_min)*.05
        x_min = x_min - x_offset
        x_max = x_max - x_offset

        xlim = (x_min,x_max)
    
    # If ylim is not specified, figure out useful ylim
    if ylim is None:
        y_min = np.nanmin(all_data["height"])
        y_max = np.nanmax(all_data["height"])
        y_offset = (y_max - y_min)*.15
        y_min = y_min - y_offset
        y_max = y_max - y_offset
        
        ylim = (y_min,y_max)
    
    # Generate plots for each residue
    plot_row = 0
    plot_column = 0
    index = 0 
    while True:

        # Update counter if we've reached right-most column
        if plot_column >= num_across:
            plot_column = 0
            plot_row += 1

        # See if we've run out of subplots
        if plot_row >= num_down:
            break
        if plot_column >= num_down:
            break

        # ax for appropriate subplot
        a = ax[plot_row,plot_column]
            
        # Set title to residue number
        if index < len(resid_list):
            a.set_title(resid_list[index])
        
        # Create pretty axes
        a.set_xlim(*xlim)
        a.set_ylim(*ylim)
        
        for s in ["right","left","top","bottom"]:
            a.spines[s].set_visible(False)
        
        # For left-most column, add y-axes
        if plot_column == 0:
            a.spines["left"].set_visible(True)
            a.set_ylabel("height")
        else:
            a.get_yaxis().set_ticks([])
        
        # For bottom row, add x-axes
        if plot_row == num_down - 1:
            a.spines["bottom"].set_visible(True)
            a.set_xlabel("time (s)")
        else:
            a.get_xaxis().set_ticks([])
        
        # Update counters
        plot_column += 1
        index += 1
        
    # -------------------------------------------------------------------------
    # Do fitting and plotting for each protein and each residues
    
    # Lists to hold output data for final df
    protein = []
    rate = []
    err = []
    residue = []
    flag = []
    
    # Loop over proteins
    for index, prot in enumerate(proteins):

        # Counters for keeping track of subplot
        plot_row = 0
        plot_column = 0
        
        # Grab protein-specific data
        data = all_data.loc[all_data["protein"] == prot]
        zero_data = zero_points.loc[zero_points["protein"] == prot]
        
        # For each residue
        for i in resid_list:
            
            # Figure out what subplot we're on
            if plot_column >= num_across:
                plot_column = 0
                plot_row += 1
            
            # Grab residue-specific data.  If there is no data for this 
            # residue for this protein, move on without plotting or fitting
            df = data.loc[data["residue"] == i]
            if len(df) == 0:
                plot_column += 1
                continue

            # Record that we are doing to fit this protein/residue pair
            protein.append(prot)
            residue.append(i)
            
            # Grab time, height, and fraction for doing fits
            X = np.array(df["time"])
            Y = np.array(df["height"])
            FX = np.array(df["fx"])
            
             # Get the zero data frame for this residue
            zero_df = zero_data.loc[zero_data["residue"] == i]
            
            # We're going to fit three models.  Make list to hold aic and model
            # name for each fit. 
            all_aic = []
            model_names = []
            
            # fit line w/ slope = 0
            lin_opt, lin_cov = curve_fit(line,X,Y,p0=[Y[0]])
            lin_model_vals = line(X, *lin_opt)

            all_aic.append(get_aic(lin_model_vals,Y,1))
            model_names.append("linear")

            # If we have actually have a peak above try_to_fit_cutoff, try exp and stretch
            if np.max(Y) > try_to_fit_cutoff:
            
                # exponential, force positive k
                try:
                    exp_opt, exp_cov = curve_fit(decay,X,Y,p0=[3,0.02],bounds=([0.5,0],np.inf))
                    exp_model_vals = decay(X, *exp_opt)
                    all_aic.append(get_aic(exp_model_vals,Y,2))
                    model_names.append("exponential")

                except RuntimeError:
                    pass

                # Stretched exponential     
                try:

                    str_opt, str_cov = curve_fit(decay_stretch,X,Y,p0=[3,0.02,1],
                                                 bounds=([0.5,0,0],
                                                         [np.inf,np.inf,np.inf]))
                    str_model_vals = decay_stretch(X, *str_opt)
                    all_aic.append(get_aic(str_model_vals,Y,3))
                    model_names.append("stretched")

                except RuntimeError:
                    pass

            # Get the model with the minimum AIC
            model = model_names[all_aic.index(min(all_aic))]

            # If linear model was best model, record it.  If the mean FX for 
            # this residue is above slow cutoff, assign it "slow" (no exchange).
            # Otherwise, assign it "fast" (exchange faster than we could
            # measure). 
            if model == "linear":
                
                rate.append(0)
                err.append(0)      
                if np.mean(FX) > slow_cutoff:
                    flag.append("slow")
                else:
                    flag.append("fast")
        
                model_values = [line(x_span, *lin_opt) for _ in range(len(x_span))]
            
            # If exponential was best model, record it
            elif model == "exponential":
                rate.append(exp_opt[1])
                err.append(np.sqrt(np.diag(exp_cov))[1])
                flag.append("exponential")
                model_values = decay(x_span, *exp_opt)
            
            # If stretched was best model, record it
            elif model == "stretched":
                rate.append(str_opt[1])
                err.append(np.sqrt(np.diag(str_cov))[1])
                flag.append("stretch")
                model_values = decay_stretch(x_span, *str_opt)
                
            # If we get here, something dreadful happened
            else:
                err = "should never get here\n"
                raise RuntimeError(err)
            
            # Load manually mapped in calls. 
            try:
                flag[-1] = manual_mapping[prot][i]
            except (KeyError,TypeError):
                pass
            
            # Figre out what to do with color in plot
            color = colors[index]
            if flag[-1] == "fast":
                color = secondary_colors[index]

            # Get subplot we're going to draw on
            a = ax[plot_row,plot_column]
            
            # Plot points and fit, if successful
            a.plot(X, Y, "ok", color=color, ms=2)
            if model_values is not None:
                a.plot(x_span,model_values,"-",color=color)
                
            if plot_zero_points:
                if len(zero_df) > 0:
                    zero_y = zero_df.height.iloc[0]
                    a.plot((x_span[0],x_span[-1]),(zero_y,zero_y),'--',color=color)
                
            # Update subplot counter
            plot_column += 1

    # Build final results data frame
    results = pd.DataFrame({"protein":protein,
                            "residue":residue,
                            "flag":flag,
                            "rate":rate,
                            "err":err})

    # Clean up fig
    fig.tight_layout()
    
    return fig, ax, results

## Load and fit exchange data

In [ ]:
manual_mapping = {"M63F":{19:"slow",
                          22:"slow",
                          40:"slow",
                          41:"slow",
                          42:"slow",
                          66:"slow",
                          78:"slow",
                          80:"fast",
                          82:"slow",
                          83:"slow",
                          84:"fast"},
                  "A9":{53:"fast",
                        79:"fast",
                        82:"slow",
                        83:"slow"}}

for p in ["A9","M63F"]:
    for i in range(46,62):
        manual_mapping[p][i] = "fast"
    
    for i in range(86,114):
        manual_mapping[p][i] = "fast"

all_files = get_file_time_index("peak-file-index.xlsx","peak-files/")
peaks = load_peak_files(all_files)
fig, ax, exchange_data = hdx_fitter_aic(peaks,manual_mapping=manual_mapping) 

fig.savefig("hdx-fit-plots.pdf")

## Construct summary data frame

In [ ]:
residues = list(exchange_data.residue.unique())
residues.sort()

out = {"residue":residues}
df = pd.DataFrame(out)
df["A9_type"] = None
df["A9_rate"] = np.nan
df["A9_err"] = np.nan
df["M63F_type"] = None
df["M63F_rate"] = np.nan
df["M63F_err"] = np.nan

for i, r in enumerate(residues):
    for prot in ["A9","M63F"]:
        mask = np.logical_and(exchange_data.residue == r,
                              exchange_data.protein == prot)
        tmp = exchange_data.loc[mask,:]
        if len(tmp) > 0:
            df[f"{prot}_type"].iloc[i] = tmp["flag"].iloc[0]
            df[f"{prot}_rate"].iloc[i] = tmp["rate"].iloc[0]
            df[f"{prot}_err"].iloc[i] = tmp["err"].iloc[0]

df.to_csv("hdx-fit-results.csv")

## Write out results to bfactor column of pdb file

In [ ]:
TO_NUMBER = {"fast":0,
             "exponential":1,
             "slow":2,
             ("fast","fast"):3,
             ("fast","exponential"):4,
             ("exponential","exponential"):5,
             ("exponential","slow"):6,
             ("slow","slow"):7,
             ("exponential","fast"):8,

             # curves without data
             (None,"exponential"):9,
             (None,"fast"):9,
             ("fast",None):9,
             None:9}
    
def to_bfactor(pdb_file,out_file,to_write):

    out = []
    with open(pdb_file,"r") as f:
        for line in f:
            if line[:4] == "ATOM":
                r = int(line[22:26])
                try:
                    number = to_write[r]
                except KeyError:
                    number = TO_NUMBER[None]

                out.append("{}{:6.2f}{}".format(line[:60],number,line[66:]))
            else:
                out.append(line)

    f = open(out_file,"w")
    f.write("".join(out))
    f.close()
                
a9 = {}
m63f = {}
compare = {}
for i in range(len(df)):

    row = df.iloc[i]
    A9_type = row.A9_type
    M63F_type = row.M63F_type

    a9[row.residue] = TO_NUMBER[A9_type]
    m63f[row.residue] = TO_NUMBER[M63F_type]
    compare[row.residue] = TO_NUMBER[(A9_type,M63F_type)]

to_bfactor("structure/1irj.pdb","structure/1irj_with-exchange_a9.pdb",a9)
to_bfactor("structure/1irj.pdb","structure/1irj_with-exchange_m63f.pdb",m63f)
to_bfactor("structure/1irj.pdb","structure/1irj_with-exchange_diff.pdb",compare)
                                  


## Generate exchange vs. residue plot

In [ ]:

fig, ax = plt.subplots(1,1,figsize=(15,2.5))

all_dG = [v for v in -0.001987*298*np.log(df.A9_rate) if not np.isinf(v)]
all_dG.extend([v for v in -0.001987*298*np.log(df.A9_rate) if not np.isinf(v)])
all_dG = np.array(all_dG)
all_dG = all_dG[np.logical_not(np.isnan(all_dG))]

min_dG = np.floor(np.min(all_dG))
max_dG = np.ceil(np.max(all_dG))


span = (max_dG - min_dG)

fast_line = min_dG - 0.5
slow_line = max_dG + 0.5

mapper = {"fast":fast_line,
          "slow":slow_line}

colors = {"fast":"firebrick",
          "exponential":"teal",
          "slow":"darkblue"}

ax.plot((-1,119),(min_dG,min_dG),"--",color="gray")
ax.plot((-1,119),(max_dG,max_dG),"--",color="gray")

for i in range(len(df)):
    row = df.iloc[i]
    
    resid = row.residue
    a9 = row.A9_type
    m63f = row.M63F_type
    
    if a9 == "exponential":
        a9_pos = -0.001987*298*np.log(row.A9_rate)
    else:
        try:
            a9_pos = mapper[a9]
        except KeyError:
            a9 = None
            a9_pos = np.nan
            
    if a9 is not None:
        ax.scatter([resid],[a9_pos],
                    s=80,facecolors="none",
                    edgecolors=colors[a9])
            
    if m63f == "exponential":
        m63f_pos = -0.001987*298*np.log(row.M63F_rate)
    else:
        try:
            m63f_pos = mapper[m63f]
        except KeyError:
            m63f = None
            m63f_pos = np.nan
            
    if m63f is not None:
        ax.scatter([resid],[m63f_pos],
                    s=80,facecolors=colors[m63f],
                    edgecolors=colors[m63f])

    
    # Drawing arrow
    if a9 and m63f:
        if m63f_pos > a9_pos:
            color = "blue"
        elif a9_pos > m63f_pos:
            color = "red"
        else:
            # If they are the same, do not draw arrow
            color = None
            continue
        
        if color:
            
            dy =  m63f_pos-a9_pos - 0.50
            if dy < 0:
                dy = 0
                
            if m63f_pos - a9_pos < 0:
                dy = m63f_pos - a9_pos + 0.50
                if dy > 0:
                    dy = 0
            
            ax.arrow(resid,a9_pos,
                     0,dy,
                     color=color,
                     lw=1,
                     head_width=1,
                     head_length=0.25)
    
fig.savefig("exchange-vs-residue-plot.pdf")



## A9 and M63F class contingency table

In [ ]:

dG_list = []
possible = ["fast","exponential","slow"]
for p1 in possible:
    for p2 in possible:
    
        a = df[np.logical_and(df.A9_type == p1,df.M63F_type == p2)]

        if p1 == "exponential" and p2 == "exponential":
            dG_list.append(-0.001987*298*np.log(a.M63F_rate/a.A9_rate))
            

        print(p1,p2,len(a))
    
        
    
print("ddG:",np.mean(dG_list),"+/-",np.std(dG_list))
    
    
